In [1]:
import os
import sys
import gc
import numpy as np
import pandas as pd
import pickle5
from tqdm import tqdm
import datetime
import shutil

from utils import (
    save_as_pkl,
    reduce_mem_usage
)

from sklearn.metrics import roc_auc_score, accuracy_score

import seaborn as sns
import matplotlib.pyplot as plt
sns.set()

from sklearn.linear_model import Ridge, RidgeClassifier
from sklearn.model_selection import KFold

pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', None)

In [2]:
# catboost

sys.path.append('/home/myaun/catboost/catboost/python-package')
import catboost
print('CAT Version', catboost.__version__)

CAT Version 0.24.2


# Read data

In [3]:
preds_vals = pd.DataFrame()
preds_names = []

In [4]:
now = datetime.datetime.now()
stacking_exp_name = f'{now:%Y%m%d%H%M}_stacking_v4_8'

In [5]:
is_save = True

In [6]:
if is_save is True:
    if not os.path.exists(f'../save/{stacking_exp_name}'):
        os.mkdir(f'../save/{stacking_exp_name}')
        os.mkdir(f'../save/{stacking_exp_name}/gbdt_model')
        os.mkdir(f'../save/{stacking_exp_name}/seq_model')

## GBDT

In [7]:
FOLD_NAME = 'vlatest_ALL_2p5M'
gbdt_run_id = '0166e411e51e48a8b6639b4da2510d61'

EXP_NAME = f'{FOLD_NAME}__CAT'
INPUT_DIR = '../save'
SAVE_NAME = f'{EXP_NAME}_{gbdt_run_id}'
sys.path.append(f'{INPUT_DIR}/{SAVE_NAME}/')
import config
use_features = config.use_features

X_val_wo_lec = pd.read_feather(f'{INPUT_DIR}/{SAVE_NAME}/X_val_wo_lec.feather')
y_val = X_val_wo_lec['answered_correctly'].values

importances = pd.read_csv(f'{INPUT_DIR}/{SAVE_NAME}/importance_0166e411e51e48a8b6639b4da2510d61_0.8031931720326341_20201209.csv', index_col=0).sort_values('feat')

EXP_NAME = f'{FOLD_NAME}__CAT'
INPUT_DIR = '../save'
SAVE_NAME = f'{EXP_NAME}_{gbdt_run_id}'

In [8]:
if is_save is True:
    GBDT_SAVE_NAME = f'{EXP_NAME}_{gbdt_run_id}'
    shutil.copy(f'{INPUT_DIR}/{GBDT_SAVE_NAME}/features/riiidFE.pkl', f'../save/{stacking_exp_name}/riiidFE.pkl')

In [9]:
mname = 'GBDT_cat1'
seeds = [
    1209,
    461224,
    20201209,
    20214646,
    46202146,
    46462021,
]
for se in seeds:
    preds_vals[f'{mname}_{se}'] = pd.read_csv(f'{INPUT_DIR}/{SAVE_NAME}/preds_val_{gbdt_run_id}_{se}.csv')['preds'].values
    if is_save is True:
        shutil.copy(f'{INPUT_DIR}/{GBDT_SAVE_NAME}/model/cat_model_{se}.pkl', f'../save/{stacking_exp_name}/gbdt_model/cat_model_{se}.pkl')

## Transformer

In [10]:
trans_run_id = 'transformer_020_8_20210106171509'

EXP_NAME = f'{FOLD_NAME}__Tran'
INPUT_DIR = '../save'
SAVE_NAME = f'{EXP_NAME}_{trans_run_id}'

mname = f'tran_{trans_run_id}'
preds_vals[f'{mname}'] = pd.read_csv(f'{INPUT_DIR}/{SAVE_NAME}/preds_val_{trans_run_id}.csv')['preds'].values
if is_save is True:
    shutil.copy(f'{INPUT_DIR}/{SAVE_NAME}/seq_model/best_weight.pt', f'../save/{stacking_exp_name}/seq_model/{mname}.pt')

In [11]:
trans_run_id = 'transformer_nariv2_1'

INPUT_DIR = '../save'
SAVE_NAME = f'{trans_run_id}'

mname = f'tran_{trans_run_id}'
preds_vals[f'{mname}'] = pd.read_csv(f'{INPUT_DIR}/{SAVE_NAME}/preds_val_{trans_run_id}.csv')['preds'].values
if is_save is True:
    shutil.copy(f'{INPUT_DIR}/{SAVE_NAME}/seq_model/best_weight.pt', f'../save/{stacking_exp_name}/seq_model/{mname}.pt')

In [12]:
scores = []
for i in range(preds_vals.shape[1]):
    scores.append([preds_vals.columns[i], roc_auc_score(y_val, preds_vals.iloc[:, i])])
pd.DataFrame(scores, columns=['mname', 'auc'])

,mname,auc
0,GBDT_cat1_1209,0.803260
1,GBDT_cat1_461224,0.803254
2,GBDT_cat1_20201209,0.803193
3,GBDT_cat1_20214646,0.802605
4,GBDT_cat1_46202146,0.803619
5,GBDT_cat1_46462021,0.802652
6,tran_transformer_020_8_20210106171509,0.797130
7,tran_transformer_nariv2_1,0.797847


In [14]:
preds_names = preds_vals.columns.tolist()

## Stacking

### catboost, 5fold kf (preds + important features 50)

In [15]:
def predict_cv_cat(train_x, train_y, kf):

    CAT_PARAMS = {
        'learning_rate': 0.04,
        'depth': 8,
        'l2_leaf_reg': 3.0,
        'bagging_temperature': 0.2,
        'border_count': 128,
        'od_type': 'Iter',
        'metric_period': 50,
        'iterations': 30000,
        'od_wait': 50,
        'random_seed': RANDOM_STATE,
        'task_type': 'GPU',
        'gpu_ram_part': 0.95, 
    }

    CAT_PARAMS['loss_function'] = 'Logloss'
    CAT_PARAMS['eval_metric'] = 'AUC'

    models = []
    preds = []
    va_idxes = []
    
    for fold_idx, (tr_idx, va_idx) in tqdm(enumerate(kf.split(preds_vals.values))):

        tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
        tr_y, va_y = train_y[tr_idx], train_y[va_idx]

        model = catboost.CatBoostClassifier(
            **CAT_PARAMS
        )
        model.fit(
            tr_x, tr_y,
            cat_features=[],
            eval_set=(va_x, va_y),
            use_best_model=True,
            verbose=100
        )
        
        pred = model.predict_proba(va_x)[:, 1]
        preds.append(pred)

        va_idxes.append(va_idx)
        models.append(model)

    va_idxes = np.concatenate(va_idxes)
    preds = np.concatenate(preds, axis=0)
    order = np.argsort(va_idxes)
    pred_train = preds[order]
    return models, pred_train

In [16]:
use_features = importances.sort_values('importance', ascending=False).head(75)['feat'].tolist()

In [17]:
seeds = [
    20201209,
    461209,
    120946
]

pred_sa = np.zeros(len(y_val))
aucs = []
models = []
for RANDOM_STATE in seeds:

    kf = KFold(n_splits=9, random_state=RANDOM_STATE, shuffle=True)

    models_s, pred_oof = predict_cv_cat(
        pd.concat([preds_vals, X_val_wo_lec[use_features]], axis=1),
        y_val,
        kf,
    )
    pred_sa += pred_oof / len(seeds)
    auc = roc_auc_score(y_val, pred_oof)
    aucs.append(auc)
    print(f'seed {RANDOM_STATE}, AUC: {auc}')
    
    models.extend(models_s)

print(sum(aucs) / len(aucs))

if is_save is True:
    stacking_name = f'catboost_with_fe_imp_sa{len(seeds)}'
    if not os.path.exists(f'../save/{stacking_exp_name}/{stacking_name}'):
        os.mkdir(f'../save/{stacking_exp_name}/{stacking_name}')
    save_as_pkl(models, f'../save/{stacking_exp_name}/{stacking_name}/stacking_cat_models.pkl')
    save_as_pkl(preds_names, f'../save/{stacking_exp_name}/{stacking_name}/stacking_cat_predsnames.pkl')
    save_as_pkl(use_features, f'../save/{stacking_exp_name}/{stacking_name}/stacking_cat_use_features.pkl')

0it [00:00, ?it/s]

0:	learn: 0.8015955	test: 0.8008785	best: 0.8008785 (0)	total: 33.7ms	remaining: 16m 51s
100:	learn: 0.8091644	test: 0.8082405	best: 0.8082405 (100)	total: 2.38s	remaining: 11m 45s
200:	learn: 0.8100987	test: 0.8088167	best: 0.8088167 (200)	total: 4.74s	remaining: 11m 43s
300:	learn: 0.8108082	test: 0.8090658	best: 0.8090658 (300)	total: 7.2s	remaining: 11m 50s
400:	learn: 0.8114804	test: 0.8092045	best: 0.8092062 (399)	total: 9.87s	remaining: 12m 8s
500:	learn: 0.8121356	test: 0.8093062	best: 0.8093062 (500)	total: 12.4s	remaining: 12m 11s
600:	learn: 0.8127711	test: 0.8093654	best: 0.8093685 (595)	total: 15.1s	remaining: 12m 19s
700:	learn: 0.8133748	test: 0.8094261	best: 0.8094264 (699)	total: 17.7s	remaining: 12m 21s
800:	learn: 0.8139533	test: 0.8094870	best: 0.8094876 (794)	total: 20.2s	remaining: 12m 17s
900:	learn: 0.8145727	test: 0.8095159	best: 0.8095194 (879)	total: 22.6s	remaining: 12m 10s
1000:	learn: 0.8151458	test: 0.8095307	best: 0.8095312 (992)	total: 25.2s	remaining: 

1it [00:36, 36.59s/it]

0:	learn: 0.8012802	test: 0.8019373	best: 0.8019373 (0)	total: 31.7ms	remaining: 15m 52s
100:	learn: 0.8090791	test: 0.8090006	best: 0.8090006 (100)	total: 2.69s	remaining: 13m 17s
200:	learn: 0.8100321	test: 0.8095171	best: 0.8095171 (200)	total: 5.41s	remaining: 13m 21s
300:	learn: 0.8107328	test: 0.8097540	best: 0.8097540 (300)	total: 7.78s	remaining: 12m 47s
400:	learn: 0.8113925	test: 0.8098842	best: 0.8098850 (399)	total: 10.2s	remaining: 12m 32s
500:	learn: 0.8120671	test: 0.8099718	best: 0.8099730 (496)	total: 12.9s	remaining: 12m 40s
600:	learn: 0.8127002	test: 0.8100303	best: 0.8100314 (598)	total: 15.6s	remaining: 12m 42s
700:	learn: 0.8133081	test: 0.8100763	best: 0.8100775 (697)	total: 18.2s	remaining: 12m 41s
800:	learn: 0.8139042	test: 0.8101053	best: 0.8101053 (799)	total: 21s	remaining: 12m 44s
900:	learn: 0.8144882	test: 0.8101328	best: 0.8101336 (891)	total: 23.5s	remaining: 12m 37s
1000:	learn: 0.8150564	test: 0.8101614	best: 0.8101651 (985)	total: 26.1s	remaining: 

2it [01:05, 31.87s/it]

...

0:	learn: 0.8014539	test: 0.8005056	best: 0.8005056 (0)	total: 31.4ms	remaining: 15m 43s
100:	learn: 0.8091606	test: 0.8081242	best: 0.8081242 (100)	total: 2.58s	remaining: 12m 45s
200:	learn: 0.8101124	test: 0.8087627	best: 0.8087627 (200)	total: 5.26s	remaining: 12m 59s
300:	learn: 0.8108035	test: 0.8090228	best: 0.8090228 (300)	total: 7.74s	remaining: 12m 43s
400:	learn: 0.8114702	test: 0.8091790	best: 0.8091790 (400)	total: 10.2s	remaining: 12m 36s
500:	learn: 0.8121306	test: 0.8092846	best: 0.8092846 (500)	total: 12.8s	remaining: 12m 33s
600:	learn: 0.8127473	test: 0.8093420	best: 0.8093420 (600)	total: 15.4s	remaining: 12m 35s
700:	learn: 0.8133348	test: 0.8093984	best: 0.8094000 (696)	total: 18.2s	remaining: 12m 38s
800:	learn: 0.8139347	test: 0.8094355	best: 0.8094382 (789)	total: 20.7s	remaining: 12m 36s
900:	learn: 0.8145339	test: 0.8094627	best: 0.8094627 (900)	total: 23.4s	remaining: 12m 36s
1000:	learn: 0.8151298	test: 0.8094842	best: 0.8094859 (992)	total: 26s	remaining: 

8it [04:16, 33.68s/it]

0:	learn: 0.8013716	test: 0.8013836	best: 0.8013836 (0)	total: 29.1ms	remaining: 14m 32s
100:	learn: 0.8090803	test: 0.8087842	best: 0.8087842 (100)	total: 2.38s	remaining: 11m 44s
200:	learn: 0.8100430	test: 0.8093792	best: 0.8093792 (200)	total: 4.91s	remaining: 12m 7s
300:	learn: 0.8107356	test: 0.8096349	best: 0.8096349 (300)	total: 7.49s	remaining: 12m 18s
400:	learn: 0.8113843	test: 0.8097847	best: 0.8097847 (400)	total: 9.93s	remaining: 12m 13s
500:	learn: 0.8120617	test: 0.8098818	best: 0.8098818 (500)	total: 12.5s	remaining: 12m 17s
600:	learn: 0.8127114	test: 0.8099457	best: 0.8099457 (600)	total: 15.2s	remaining: 12m 23s
700:	learn: 0.8133217	test: 0.8099849	best: 0.8099849 (700)	total: 17.8s	remaining: 12m 25s
800:	learn: 0.8139195	test: 0.8100227	best: 0.8100239 (789)	total: 20.6s	remaining: 12m 29s
bestTest = 0.8100401759
bestIteration = 828
Shrink model to first 829 iterations.


9it [04:40, 31.19s/it]


seed 120946, AUC: 0.8100139453990907
0.8100176037455347
